In [1]:
from sympy import *
import numpy as np 
init_printing()

# Definitions
eta, dx, dz = symbols( 'eta, dx, dz' )
D11, D12, D13, D21, D22, D23 ,D31, D32, D33 = symbols('D11, D12, D13, D21, D22, D23 ,D31, D32, D33')
Exx, Ezz, Gxz = symbols('Exx, Ezz, Gxz')
stress_labels = ['Txx', 'Tzz', 'Txz']

D11 = 2*eta; D12 = 0*eta; D13 = 0*eta;
D21 = 0*eta; D22 = 2*eta; D23 = 0*eta;
D31 = 0*eta; D32 = 0*eta; D33 = 1*eta;

Dv = Matrix( [ [D11, D12, D13], [D21, D22, D23], [D31, D32, D33]  ] )
div = Exx+Ezz
Ed  = Matrix( [ [Exx-1/3*div], [Ezz-1/3*div], [Gxz] ] )
T  = Dv*Ed;
for i in range(len(stress_labels)):
    print(stress_labels[i] + ' = ' + ccode( T[i]) + ';' )


Txx = 2*eta*(0.666666666666667*Exx - 0.333333333333333*Ezz);
Tzz = 2*eta*(-0.333333333333333*Exx + 0.666666666666667*Ezz);
Txz = Gxz*eta;


In [3]:
# Stencil x
uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE = symbols('uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE')

dofs = Matrix([uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE])
#----------------------------------------#
divW = 1/dx*(uC-uW) + 1/dz*(vNW-vSW)
ExxW = 1/dx*(uC-uW) - 1/3*divW;
EzzW = 0;
GxzW = 0
EdW  = Matrix([[ExxW], [EzzW], [GxzW]])
TW   = Dv*EdW
TxxW = TW[0].subs(eta,'etaW')
#TxxW = 2* etaW* ExxW
#----------------------------------------#
divE = 1/dx*(uE-uC) + 1/dz*(vNE-vSE)
ExxE = 1/dx*(uE-uC) - 1/3*divE
EzzE = 0;
GxzE = 0
EdE  = Matrix([[ExxE], [EzzE], [GxzE]])
TE   = Dv*EdE
TxxE = TE[0].subs(eta,'etaE')
#----------------------------------------#
ExxS = 0
EzzS = 0
GxzS = 1/dz*(uC-uS) + 1/dx*(vSE-vSW)
EdS  = Matrix([[ExxS], [EzzS], [GxzS]])
TS   = Dv*EdS
TxzS = TS[2].subs(eta,'etaS')
#----------------------------------------#
ExxN = 0
EzzN = 0
GxzN = 1/dz*(uN-uC) + 1/dx*(vNE-vNW)
EdN  = Matrix([[ExxN], [EzzN], [GxzN]])
TN   = Dv*EdN
TxzN = TN[2].subs(eta,'etaN')
#----------------------------------------#
Fx = 1/dx*(TxxE - TxxW) + 1/dz*(TxzN - TxzS)  
Fx = -Fx
for i in range(len(dofs)):
    cUc = Fx.diff(dofs[i])
    print(str(dofs[i]) + ' = ' +  ccode(cUc) + ';')

uW = -1.33333333333333*etaW/pow(dx, 2);
uC = -(-etaN/dz - etaS/dz)/dz - (-1.33333333333333*etaE/dx - 1.33333333333333*etaW/dx)/dx;
uE = -1.33333333333333*etaE/pow(dx, 2);
uS = -etaS/pow(dz, 2);
uN = -etaN/pow(dz, 2);
vSW = -etaS/(dx*dz) + 0.666666666666667*etaW/(dx*dz);
vSE = -0.666666666666667*etaE/(dx*dz) + etaS/(dx*dz);
vNW = etaN/(dx*dz) - 0.666666666666667*etaW/(dx*dz);
vNE = 0.666666666666667*etaE/(dx*dz) - etaN/(dx*dz);


In [4]:
# Stencil z
vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE = symbols('vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE')
dofs = Matrix([vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE])
#----------------------------------------#
divS = 1/dz*(vC-vS) + 1/dx*(uSE-uSW)
ExxS = 0
EzzS = 1/dz*(vC-vS) - 1/3*divS;
GxzS = 0
EdS  = Matrix([[ExxS], [EzzS], [GxzS]])
TS   = Dv*EdS
TzzS = TS[1].subs(eta,'etaS')
#----------------------------------------#
divN = 1/dz*(vN-vC) + 1/dx*(uNE-uNW)
ExxN = 0
EzzN = 1/dz*(vN-vC)  - 1/3*divN;
GxzN = 0
EdN  = Matrix([[ExxN], [EzzN], [GxzN]])
TN   = Dv*EdN
TzzN = TN[1].subs(eta,'etaN')
#----------------------------------------#
ExxW = 0
EzzW = 0
GxzW = 1/dz*(uNW-uSW) + 1/dx*(vC-vW)
EdW  = Matrix([[ExxW], [EzzW], [GxzW]])
TW   = Dv*EdW
TxzW = TW[2].subs(eta,'etaW')
# #----------------------------------------#
ExxE = 0
EzzE = 0
GxzE = 1/dz*(uNE-uSE) + 1/dx*(vE-vC)
EdE  = Matrix([[ExxE], [EzzE], [GxzE]])
TE   = Dv*EdE
TxzE = TE[2].subs(eta,'etaE')
#----------------------------------------#
Fz = 1/dz*(TzzN - TzzS) + 1/dx*(TxzE - TxzW)  
Fz = -Fz
for i in range(9):
    cVc = Fz.diff(dofs[i])
    print(str(dofs[i]) + ' = ' +  ccode(cVc) + ';') #print(cUc)

vW = -etaW/pow(dx, 2);
vC = -(-1.33333333333333*etaN/dz - 1.33333333333333*etaS/dz)/dz - (-etaE/dx - etaW/dx)/dx;
vE = -etaE/pow(dx, 2);
vS = -1.33333333333333*etaS/pow(dz, 2);
vN = -1.33333333333333*etaN/pow(dz, 2);
uSW = 0.666666666666667*etaS/(dx*dz) - etaW/(dx*dz);
uSE = etaE/(dx*dz) - 0.666666666666667*etaS/(dx*dz);
uNW = -0.666666666666667*etaN/(dx*dz) + etaW/(dx*dz);
uNE = -etaE/(dx*dz) + 0.666666666666667*etaN/(dx*dz);
